In [1]:
! pip install stop_words

You are using pip version 8.0.3, however version 8.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
import collections

s1 = "The quick brown fox"
s2 = "Brown fox jumps over the jumps jumps jumps"
s3 = "The the the lazy dog elephant."
s4 = "The the the the the dog peacock lion tiger elephant"

docs = collections.OrderedDict()
docs["s1"] = s1
docs["s2"] = s2
docs["s3"] = s3
docs["s4"] = s4
for k, v in docs.items():
    print(k,v)
    

s1 The quick brown fox
s2 Brown fox jumps over the jumps jumps jumps
s3 The the the lazy dog elephant.
s4 The the the the the dog peacock lion tiger elephant


In [2]:
len(docs)

4

**Function to make Corpus Matrix**

Function returns a list, 0th element is a 3 dimensional array, each element corresponds to a document in the corpus where the columns are the unique words in that document and the rows are the words in that document (stop words removed).  Second element in list is the column names for each document.

In [3]:
import string
import stop_words
import numpy as np
from stop_words import get_stop_words

stopWords = get_stop_words('english')

#Function to make document, word matricies for LDA#
def make_word_matrix(corpus):

    #Define list to store corpus data#
    c = []
    #Define list to store order of words for each document#
    wordOrder = []
    #Define table to remove punctuation
    table = dict.fromkeys(map(ord, string.punctuation))

    M = len(corpus)
    #For each document in docs, caculate frequency of the words#
    for i in corpus:
        #Remove punctuation 
        text = docs[i].translate(table)
        #Splits string by blankspace and goes to lower case#
        words = text.lower().split()

        #Remove stop words#
        text = [word for word in words if word not in stopWords]

        #Find total number of words in each document#
        N = len(text)

        #Find number of unique words in each document#
        Vwords = list(set(text))
        wordOrder.append(Vwords)

    #Find unique words in the corpus, this is the vocabulary#    
    wordOrder = list(set(x for l in wordOrder for x in l))
    wordOrder = sorted(wordOrder)

    #Find the number of unique words in the corpus vocabulary#
    V = len(wordOrder)

    #For each document in docs, caculate frequency of the words#
    for i in corpus:
        #Remove punctuation 
        text = docs[i].translate(table)
        #Splits string by blankspace and goes to lower case#
        words = text.lower().split()

        #Remove stop words#
        text = [word for word in words if word not in stopWords]

        #Find total number of words in each document#
        N = len(text)

        #Create matrix to store words for each document#
        wordsMat = np.zeros((N, V))
        count = 0
        for word in text:
            v = wordOrder.index(word)
            wordsMat[count, v] = 1
            count = count + 1
        c.append(wordsMat)

    return [c, wordOrder, M] 

In [4]:
corpusMatrix = make_word_matrix(docs)
corpusMatrix

[[array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
         [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]]),
  array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]]),
  array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]),
  array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
         [ 0.,  0.

**Variational Inference**

In [5]:
import numpy as np
import scipy
from scipy import special

**E-Step:** This function uses variational inference to perform the E step in the EM algorithm to estimate the paramteters in the model.  The output of this function are the matricies gamma and phi, where gamma (k vector) is the Dirichlet paramteters and the matrix phi (N x k, where k is the number of topics) are the multinomial paramters.  See page 1004 of paper for derivation.

In [6]:
def Estep(k, d, alpha, beta, corpusMatrix, tol):    
    
    #storing the total number of words and the number of unique words
    N = corpusMatrix[0][d].shape[0]
    V = corpusMatrix[0][d].shape[1]
    
    #initialize phi and gamma
    oldPhi  = np.full(shape = (N,k), fill_value = 1/k)
    gamma = alpha + N/k
    newPhi = oldPhi
    converge = 0 
    
    
    count = 0
    
    while converge == 0:
        newPhi  = np.zeros(shape = (N,k))
        for n in range(0, N):
            for i in range(0,k):
                newPhi[n,i] = (beta[i, list(corpusMatrix[0][d][n,:]).index(1)])*np.exp(scipy.special.psi(gamma[i]))
        newPhi = newPhi/np.sum(newPhi, axis = 1)[:, np.newaxis] #normalizing the rows of new phi

        for i in range(0,k):
            gamma[i] = alpha[i] + np.sum(newPhi[:, i]) #updating gamma


        criteria = (1/(N*k)*np.sum((newPhi - oldPhi)**2))**0.5
        if criteria < tol:
            converge = 1
        else:
            oldPhi = newPhi
            count = count +1
            converge = 0
    return (newPhi, gamma)

**Parameter Estimation**

**M Step:** In the E step above, we maximized a lower bound with respect to gamma and phi, and in the M step, for fixed values of these variational parameters, we maximize the lower bound of the log likelihood with repsect to alpha and beta to update these values (combined, these two steps give approximate empirical Bayes estimates for the LDA model).  See pg. 1006 and appendix A.2 for derivation.  

The alphaUpdate() function uses the linear Newton-Rhapson method to update the Dirichlet parameters, alpha, while the Mstep() function maximizes for alpha and beta.

In [92]:
#Update alpha using linear Newton-Rhapson Method#
def alphaUpdate(k, M, alphaOld, gamma, tol):
    h = np.zeros(k)
    g = np.zeros(k)
    alphaNew = np.zeros(k)

    converge = 0
    while converge == 0:
        for i in range(0, k):
            docSum = 0
            for d in range(0, M):
                docSum += scipy.special.psi(gamma[d][i]) - scipy.special.psi(np.sum(gamma[d]))
            g[i] = M*(scipy.special.psi(sum(alphaOld)) - scipy.special.psi(alphaOld[i])) + docSum
            h[i] = M*scipy.special.polygamma(1, alphaOld[i])
        z =  -scipy.special.polygamma(1, np.sum(alphaOld))
        c = np.sum(g/h)/(1/z + np.sum(1/h))
        step = (g - c)/h
        if np.linalg.norm(step) < tol:
            converge = 1
        else:
            converge = 0
            alphaNew = alphaOld + step
            alphaOld = alphaNew

    return alphaNew

In [98]:
def Mstep(k, M, phi, gamma, alphaOld, corpusMatrix, tol):
    #Calculate beta#
    V = corpusMatrix[0][0].shape[1]
    beta = np.zeros(shape = (k,V))

    for i in range(0,k):
        for j in range(0,V):
            wordSum = 0
            for d in range(0,M):
                Nd = corpusMatrix[0][d].shape[0]
                for n in range(0, Nd):
                    wordSum += phi[d][n,i]*corpusMatrix[0][d][n,j]
            beta[i,j] = wordSum
    #Normalize the rows of beta#
    beta = beta/np.sum(beta, axis = 1)[:, np.newaxis]
    
    ##Update ALPHA##
    alphaNew = alphaUpdate(k, M, alphaOld, gamma, tol)
    return(alphaNew, beta)



**LDA Function:**
Finally, we implement the entire Latent Dirichlet Allocation method in the LDA function, which takes as its arguments k (the number of topics), D (the number of documents in the corpus), a corpus matrix (the output from make_word_matrix above) and a tolerance (which sets the convergence criteria for the while loops).  For each document d, the function runs until the alpha or beta parameters converge, by first running the E step and then the M step for each document separately.  The final values of phi, gamma, alpha and beta are returned for all D documents in a list.

In [106]:
#k = number of topics, D = number of documents#
#corpus matrix is output of make_word_matrix# 
def LDA(k, M, corpusMatrix, tol):

    
    output = []
    
    converge = 0
    #initialize alpha and beta for first iteration
    alphaOld = 10*np.random.rand(k)
    V = corpusMatrix[0][0].shape[1]
    betaOld = np.random.rand(k, V)
    betaOld = betaOld/np.sum(betaOld, axis = 1)[:, np.newaxis]
    
    while converge == 0:
        phi = []
        gamma = []
        #looping through the number of documents
        for d in range(0,M): #M is the number of documents
            phiT, gammaT = Estep(k, d, alphaOld, betaOld, corpusMatrix, tol)
            phi.append(phiT)
            gamma.append(gammaT)
            
        alphaNew, betaNew = Mstep(k, M, phi, gamma, alphaOld, corpusMatrix, tol)
        if np.linalg.norm(alphaOld - alphaNew) < tol or np.linalg.norm(betaOld - betaNew) < tol:
            converge =1
        else: 
            converge =0
            alphaOld = alphaNew
            betaOld = betaNew
    output.append([phi, gamma, alphaNew, betaNew])
        
    return output

In [107]:
LDA(2, 4, corpusMatrix, 1)

[[[array([[ 0.63798875,  0.36201125],
          [ 0.95899633,  0.04100367],
          [ 0.07557093,  0.92442907]]), array([[ 0.93905555,  0.06094445],
          [ 0.05110487,  0.94889513],
          [ 0.44537001,  0.55462999],
          [ 0.44537001,  0.55462999],
          [ 0.44537001,  0.55462999],
          [ 0.44537001,  0.55462999]]), array([[ 0.99099218,  0.00900782],
          [ 0.80422994,  0.19577006],
          [ 0.03317097,  0.96682903]]), array([[ 0.75026263,  0.24973737],
          [ 0.74538573,  0.25461427],
          [ 0.77060265,  0.22939735],
          [ 0.9852316 ,  0.0147684 ],
          [ 0.0244761 ,  0.9755239 ]])],
  [array([ 7.23833724,  2.02786683]),
   array([ 8.3374217 ,  3.92878237]),
   array([ 7.39417433,  1.87202975]),
   array([ 8.84173995,  2.42446413])],
  array([ 4.64425948,  1.16868581]),
  array([[ 0.1987791 ,  0.16279884,  0.00603726,  0.0132665 ,  0.18657077,
           0.10378459,  0.08070365,  0.07806273,  0.06681526,  0.1031813 ],
         [ 0.

## Future Steps

- Compare output to Python package
- Test on corpus in paper
- Model topics on different corpus
- Time and optimize (use Cython, quite slow now)
- Run collaborative filtering
- Compare to Gibbs sampling